In [78]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html

In [24]:
df = pd.read_csv('Asthma RPM State Coverage.csv')
df

,US States,Total Asthma Population,Adult Asthma Population Number,Adult Asthma Population Percent,Child Asthma \nPopulation Number,Child Asthma \nPopulation Percent,Total Asthma \nAttacks Number,Total Asthma \nAttacks Percent,Adult Asthma Attacks Number,Adult Asthma Attacks Percent,...,Largest Medicaid MCO Plan Market Share,Second Largest Medicaid MCO Plan Name,Second Largest Medicaid MCO Plan Market Share,Largest Private Health Insurer Name,Largest Private Health Insurer Market Share,Largest Private Health Insurer Coverage,Second Largest Private Health Insurer Name,Second Largest Private Health Insurer Market Share,Second Largest Private Health Insurer Coverage,Unnamed: 37
0,California,"3,313,415","2,848,466",9.3%,"464,949",5.3%,"1,590,439",48%,"1,321,688",46.4%,...,17.98%,Health Net Community Solutions (Centene),11.50%,Kaiser,48%,Kaiser covers nebulizers and supplies for pedi...,Blue Shield,17%,Not specified,\n\n
1,Texas,"2,148,148","1,617,392",7.4%,"530,756",7.1%,"975,259",45.4%,"722,974",44.7%,...,24.63%,Amerigroup Texas (Elevance),19.08%,HCSC (BCBS Texas),38%,"BCBS Texas provides coverage for codes 99454, ...",UnitedHealth,20%,Not specified,\n\n
2,New York,"1,778,160","1,449,094",9.5%,"329,066",8.0%,"789,503",44.4%,"620,212",42.8%,...,32.14%,HealthFirst,22.98%,UnitedHealth,28%,UnitedHealthcare provides coverage for codes 9...,Anthem,17%,Not specified,\n\n
3,Florida,"1,572,585","1,276,625",7.3%,"295,960",6.9%,"798,873",50.8%,"653,632",51.2%,...,42.72%,Humana Medical Plan,16.94%,BCBS FL,38%,Both Follow medicare CPT codes for RPM coverage,UnitedHealth,19%,UnitedHealthcare provides coverage for codes 9...,\n\n
4,Pennsylvania,"1,272,907","1,032,246",10.2%,"240,661",9.0%,"603,358",47.4%,"467,607",45.3%,...,19.28%,Keystone First (Independence Health Group),18.45%,Highmark (BCBS),29%,Does not cover,CVS Health,19%,Not specified,\n\n
5,Ohio,"1,080,631","895,631",9.9%,"185,000",7.1%,"480,881",44.5%,"398,556",44.5%,...,49.34%,Buckeye Health Plan (Centene),15.78%,Anthem,34%,Not specified,Medical Mutual,26%,Not specified,\n\n
6,Michigan,"1,027,498","859,534",11.0%,"167,964",7.8%,"437,714",42.6%,"361,864",42.1%,...,25.15%,Molina Healthcare,17.64%,BCBS MI,68%,"BCBS MI provides coverage for codes 99453, 994...",Spectrum Health,12%,Not Specified,\n\n
7,Illinois,"1,008,847","832,244",8.5%,"176,603",6.3%,"462,052",45.8%,"360,362",43.3%,...,31.16%,BCBS Illinois,24.64%,HCSC (BCBS Illinois),61%,Does not cover,UnitedHealth,14%,UnitedHealthcare provides coverage for codes 9...,\n\n
8,Georgia,"979,245","736,912",9.0%,"242,333",9.6%,"477,382",48.75%,"335,295",45.5%,...,51.10%,Amerigroup Community Care (Elevance),27.74%,Anthem,38%,Not specified,CVS Health,14%,Not Specified,\n\n
9,North Carolina,"776,970","645,784",7.8%,"131,186",5.7%,"382,269",49.2%,"308,039",47.7%,...,26.17%,UnitedHealthcare Community Plan of NC,21.73%,BCBS NC,53%,"BCBS NC provides coverage for codes 98976, 994...",UnitedHealth,17%,UnitedHealthcare provides coverage for codes 9...,\n\n


In [86]:
# Melt the DataFrame to have 'State' as a column and 'Adult' and 'Child' as values
melted_df = pd.melt(df, id_vars=['US States'], value_vars=['Adult Asthma Population Number', 'Child Asthma \nPopulation Number'],
                    var_name='Age Group', value_name='Population')

# Remove commas and convert 'Population' to numeric for correct counting
melted_df['Population'] = pd.to_numeric(melted_df['Population'].str.replace(',', ''), errors='coerce')

# Initialize the Dash app
app = dash.Dash(__name__)


# Define the app layout
app.layout = html.Div([
    html.H1("Asthma Population by State"),
    html.Div([
        dcc.Dropdown(
            id='top-n-dropdown',
            options=[
                {'label': f'Top {i} States', 'value': i} for i in range(5, df.shape[0] + 1, 5)
            ],
            value=5,
            multi=False,
            style={'width': '50%'}
        ),
    ], style={'marginTop': '20px'}),
    html.Div([
        dcc.Graph(id='stacked-bar-chart', style={'marginTop': '20px'}),
    ]),
])

# Define callback to update the chart based on the selected top-n value
@app.callback(
    dash.dependencies.Output('stacked-bar-chart', 'figure'),
    [dash.dependencies.Input('top-n-dropdown', 'value')]
)
def update_chart(top_n):
    sorted_df = df.copy()
    sorted_df['Total Asthma Population'] = sorted_df['Total Asthma Population'].str.replace(',', '').astype(int)
    sorted_df = sorted_df.sort_values(by='Total Asthma Population', ascending=False).head(top_n)
    
    melted_sorted_df = pd.melt(sorted_df, id_vars=['US States'], value_vars=['Adult Asthma Population Number', 'Child Asthma \nPopulation Number'],
                               var_name='Age Group', value_name='Population')

    melted_sorted_df['Population'] = pd.to_numeric(melted_sorted_df['Population'].str.replace(',', ''), errors='coerce')

    fig = px.bar(
        melted_sorted_df,
        x='US States',
        y='Population',
        color='Age Group',
        # title=f'Top {top_n} States - Asthma Population by Age Group',
        labels={'Population': 'Population'},
        color_discrete_map={'Adult Asthma Population Number': 'blue', 'Child Asthma Population Number': 'orange'},
        barmode='stack'
    )
    
    # Adjust the figure height and rotate x-axis labels
    fig.update_layout(height=600, width=1200, xaxis_tickangle=-45)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
